# Import Databases 

In [78]:
from pandas import ExcelWriter

import brightway2 as bw
#import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
#import seaborn as sns
import scipy as sp
import time

%load_ext nb_black

The nb_black extension is already loaded. To reload it, use:
  %reload_ext nb_black


<IPython.core.display.Javascript object>

In [79]:
bw.projects.set_current("Ammonia")  # setup and name project

<IPython.core.display.Javascript object>

In [80]:
bw.bw2setup()

Biosphere database already present!!! No setup is needed


<IPython.core.display.Javascript object>

In [81]:
if "EC" not in bw.databases:
    link = "D:\ecoinvent3.8cutoffecoSpold02\datasets"
    ei = bw.SingleOutputEcospold2Importer(link, "EC", use_mp=False)
    ei.apply_strategies()
    ei.statistics()
    ei.write_database()

<IPython.core.display.Javascript object>

In [5]:
len(bw.Database("EC"))  # display number of act in DB

19567

<IPython.core.display.Javascript object>

In [6]:
eidb = bw.Database("EC")
type(eidb)

bw2data.backends.peewee.database.SQLiteBackend

<IPython.core.display.Javascript object>

In [7]:
biodb = bw.Database("biosphere3")
type(eidb)

bw2data.backends.peewee.database.SQLiteBackend

<IPython.core.display.Javascript object>

# Building LCA Matrices

In [8]:
structured_array = np.load(eidb.filepath_processed())
pd.DataFrame(structured_array).head()

,input,output,row,col,type,uncertainty_type,amount,loc,scale,shape,minimum,maximum,negative
0,1,4710,4294967295,4294967295,2,0,3.106673e-12,3.106673e-12,NaN,NaN,NaN,NaN,False
1,1,4711,4294967295,4294967295,2,0,1.464257e-13,1.464257e-13,NaN,NaN,NaN,NaN,False
2,1,4712,4294967295,4294967295,2,0,8.955004e-13,8.955004e-13,NaN,NaN,NaN,NaN,False
3,1,4713,4294967295,4294967295,2,0,1.390754e-11,1.390754e-11,NaN,NaN,NaN,NaN,False
4,1,4714,4294967295,4294967295,2,0,1.439336e-13,1.439336e-13,NaN,NaN,NaN,NaN,False


<IPython.core.display.Javascript object>

In [9]:
pd.Series(bw.mapping).head()

biosphere3  38a622c6-f086-4763-a952-7c6b3b1c42ba        1
            541a823c-0aad-4dc4-9123-d4af4647d942        2
            8cbaa905-41b0-4327-8403-bf1c8eb25429        3
            f681eb3c-854a-4f78-bcfe-76dfbcf9df3c        4
            a0fec60d-3f74-48bf-a2d2-58c30fc13e53        5
                                                    ...  
EC          7ae91b7ca74722aa3b31eb8c38e6c5de        11107
            e461fe6593f7cdf4ceee034de532b148        11108
            1ab9438d506e7a3f192a0295b5d06502        11109
            1a148e5277d73feda0bb7a293999f07d        11110
            648e3a11c5764d93dcac73bd71c0be18        11111
Length: 11111, dtype: int64

<IPython.core.display.Javascript object>

In [10]:
eidb.metadata  # 数据库类型

{'overwrite': False,
 'format': 'Ecoinvent XML',
 'depends': ['biosphere3'],
 'backend': 'sqlite',
 'number': 19565,
 'modified': '2022-12-25T19:46:47.325940',
 'searchable': True,
 'processed': '2022-12-25T21:31:59.327002'}

<IPython.core.display.Javascript object>

In [116]:
bw.databases

Databases dictionary with 12 object(s):
	AWE PV
	AWE Wind
	CG
	CG CCS
	EC
	PEM PV
	PEM Wind
	SMR
	SMR_CCS
	SOEC PV
	SOEC Wind
	biosphere3

<IPython.core.display.Javascript object>

# Setup Foreground Database 

In [72]:
process = [
    act
    for act in eidb
    if "treatment of wastewater, average, capacity 1E9l/year" in act["name"]
    and "RoW" in act["location"]
][0]
process

'treatment of wastewater, average, capacity 1E9l/year' (cubic meter, RoW, None)

<IPython.core.display.Javascript object>

In [ ]:
process.as_dict()  #search the code of activity

In [105]:
# Setup Foreground Database for SMR
CG = bw.ExcelImporter(
    r"D:\2 Master\2 Paper\2 Ammonia\2 Python\4 code\1 data\input\CG_import.xlsx"
)
CG.apply_strategies(verbose=False)
CG.match_database("EC", fields=("name", "unit", "location", "code"))
CG.statistics()
CG.write_excel()
CG.write_database()

Extracted 1 worksheets in 0.02 seconds
Applying strategy: link_iterable_by_fields


Writing activities to SQLite3 database:


8 datasets
41 exchanges
0 unlinked exchanges
  
Wrote matching file to:
C:\Users\summer\AppData\Local\pylca\Brightway3\Ammonia.454b2d1311d3cfbe8c10a38c923bf0c5\output\db-matching-CG.xlsx


0% [########] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00


Title: Writing activities to SQLite3 database:
  Started: 01/12/2023 13:06:24
  Finished: 01/12/2023 13:06:24
  Total time elapsed: 00:00:00
  CPU %: 0.00
  Memory %: 2.50
Created database: CG


<IPython.core.display.Javascript object>

In [106]:
# Setup Foreground Database for SMR
CG_CCS = bw.ExcelImporter(
    r"D:\2 Master\2 Paper\2 Ammonia\2 Python\4 code\1 data\input\CG_CCS_import.xlsx"
)
CG_CCS.apply_strategies(verbose=False)
CG_CCS.match_database("EC", fields=("name", "unit", "location", "code"))
CG_CCS.statistics()
CG_CCS.write_excel()
CG_CCS.write_database()

Extracted 1 worksheets in 0.03 seconds
Applying strategy: link_iterable_by_fields


Writing activities to SQLite3 database:


10 datasets
71 exchanges
0 unlinked exchanges
  
Wrote matching file to:
C:\Users\summer\AppData\Local\pylca\Brightway3\Ammonia.454b2d1311d3cfbe8c10a38c923bf0c5\output\db-matching-CG-CCS.xlsx


0% [##########] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00


Title: Writing activities to SQLite3 database:
  Started: 01/12/2023 13:06:48
  Finished: 01/12/2023 13:06:48
  Total time elapsed: 00:00:00
  CPU %: 208.30
  Memory %: 2.51
Created database: CG CCS


<IPython.core.display.Javascript object>

In [107]:
# Setup Foreground Database for SMR
SMR = bw.ExcelImporter(
    r"D:\2 Master\2 Paper\2 Ammonia\2 Python\4 code\1 data\input\SMR_import.xlsx"
)
SMR.apply_strategies(verbose=False)
SMR.match_database("EC", fields=("name", "unit", "location", "code"))
SMR.statistics()
SMR.write_excel()
SMR.write_database()

Extracted 1 worksheets in 0.04 seconds
Applying strategy: link_iterable_by_fields


Writing activities to SQLite3 database:


15 datasets
57 exchanges
0 unlinked exchanges
  
Wrote matching file to:
C:\Users\summer\AppData\Local\pylca\Brightway3\Ammonia.454b2d1311d3cfbe8c10a38c923bf0c5\output\db-matching-SMR.xlsx


0% [###############] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00


Title: Writing activities to SQLite3 database:
  Started: 01/12/2023 13:07:13
  Finished: 01/12/2023 13:07:13
  Total time elapsed: 00:00:00
  CPU %: 0.00
  Memory %: 2.51
Created database: SMR


<IPython.core.display.Javascript object>

In [108]:
# Setup Foreground Database for SMR_CCS
SMR_CCS = bw.ExcelImporter(
    r"D:\2 Master\2 Paper\2 Ammonia\2 Python\4 code\1 data\input\SMR_CCS_import.xlsx"
)
SMR_CCS.apply_strategies(verbose=False)
SMR_CCS.match_database("EC", fields=("name", "unit", "location", "code"))
SMR_CCS.statistics()
SMR_CCS.write_excel()
SMR_CCS.write_database()

Extracted 1 worksheets in 0.04 seconds
Applying strategy: link_iterable_by_fields


Writing activities to SQLite3 database:


16 datasets
81 exchanges
0 unlinked exchanges
  
Wrote matching file to:
C:\Users\summer\AppData\Local\pylca\Brightway3\Ammonia.454b2d1311d3cfbe8c10a38c923bf0c5\output\db-matching-SMR_CCS.xlsx


0% [################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00


Title: Writing activities to SQLite3 database:
  Started: 01/12/2023 13:07:37
  Finished: 01/12/2023 13:07:37
  Total time elapsed: 00:00:00
  CPU %: 0.00
  Memory %: 2.45
Created database: SMR_CCS


<IPython.core.display.Javascript object>

In [104]:
# Setup Foreground Database for green ammonia
# SOEC_Wind
SOEC_Wind = bw.ExcelImporter(
    r"D:\2 Master\2 Paper\2 Ammonia\2 Python\4 code\1 data\input\SOEC_Wind_import.xlsx"
)

# SOEC_PV
SOEC_PV = bw.ExcelImporter(
    r"D:\2 Master\2 Paper\2 Ammonia\2 Python\4 code\1 data\input\SOEC_PV_import.xlsx"
)

# PEM_Wind
PEM_Wind = bw.ExcelImporter(
    r"D:\2 Master\2 Paper\2 Ammonia\2 Python\4 code\1 data\input\PEM_Wind_import.xlsx"
)

# PEM_PV
PEM_PV = bw.ExcelImporter(
    r"D:\2 Master\2 Paper\2 Ammonia\2 Python\4 code\1 data\input\PEM_PV_import.xlsx"
)

# AEM_Wind
AWE_Wind = bw.ExcelImporter(
    r"D:\2 Master\2 Paper\2 Ammonia\2 Python\4 code\1 data\input\AWE_Wind_import.xlsx"
)

# AEM_PV
AWE_PV = bw.ExcelImporter(
    r"D:\2 Master\2 Paper\2 Ammonia\2 Python\4 code\1 data\input\AWE_PV_import.xlsx"
)

Green_databases = [SOEC_Wind, SOEC_PV, PEM_Wind, PEM_PV, AWE_Wind, AWE_PV]

for database in Green_databases:
    database.apply_strategies(verbose=False)
    database.match_database("EC", fields=("name", "unit", "location", "code"))
    database.statistics()
    database.write_excel()
    database.write_database(verbose=False)

Extracted 1 worksheets in 0.08 seconds
Extracted 1 worksheets in 0.04 seconds
Extracted 1 worksheets in 0.03 seconds
Extracted 1 worksheets in 0.03 seconds
Extracted 1 worksheets in 0.03 seconds
Extracted 1 worksheets in 0.03 seconds
Applying strategy: link_iterable_by_fields


Writing activities to SQLite3 database:


12 datasets
64 exchanges
0 unlinked exchanges
  
Wrote matching file to:
C:\Users\summer\AppData\Local\pylca\Brightway3\Ammonia.454b2d1311d3cfbe8c10a38c923bf0c5\output\db-matching-SOEC-Wind.xlsx


0% [############] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00


Title: Writing activities to SQLite3 database:
  Started: 01/12/2023 13:02:50
  Finished: 01/12/2023 13:02:50
  Total time elapsed: 00:00:00
  CPU %: 0.00
  Memory %: 2.49
Created database: SOEC Wind
Applying strategy: link_iterable_by_fields


...

Writing activities to SQLite3 database:


11 datasets
53 exchanges
0 unlinked exchanges
  
Wrote matching file to:
C:\Users\summer\AppData\Local\pylca\Brightway3\Ammonia.454b2d1311d3cfbe8c10a38c923bf0c5\output\db-matching-AWE-PV.xlsx


0% [###########] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00


Title: Writing activities to SQLite3 database:
  Started: 01/12/2023 13:04:52
  Finished: 01/12/2023 13:04:52
  Total time elapsed: 00:00:00
  CPU %: 97.70
  Memory %: 2.51
Created database: AWE PV


<IPython.core.display.Javascript object>

In [117]:
bw.databases

Databases dictionary with 12 object(s):
	AWE PV
	AWE Wind
	CG
	CG CCS
	EC
	PEM PV
	PEM Wind
	SMR
	SMR_CCS
	SOEC PV
	SOEC Wind
	biosphere3

<IPython.core.display.Javascript object>

In [118]:
# instantiate database object
# databases for grey and blue ammonia
CG_import = bw.Database("CG")
CG_CCS_import = bw.Database("CG CCS")
SMR_import = bw.Database("SMR")
SMR_CCS_import = bw.Database("SMR_CCS")

# databases for green ammonia
SOEC_Wind_import = bw.Database("SOEC Wind")
SOEC_PV_import = bw.Database("SOEC PV")
PEM_Wind_import = bw.Database("PEM Wind")
PEM_PV_import = bw.Database("PEM PV")
AWE_Wind_import = bw.Database("AWE Wind")
AWE_PV_import = bw.Database("AWE PV")

<IPython.core.display.Javascript object>

In [119]:
list(AWE_Wind_import)

['operation_hydrogen' (kilogram, GLO, None),
 'manufacturing_ammonia' (pcs, GLO, None),
 'operation' (pcs, GLO, None),
 'manufacturing_nitrogen' (pcs, GLO, None),
 'operation_nitrogen' (kilogram, GLO, None),
 'manufacturing' (pcs, GLO, None),
 'AWE Wind' (pcs, GLO, None),
 'NH3 synthesis catalyst' (kilogram, GLO, None),
 'manufacturing_BoP' (pcs, GLO, None),
 'manufacturing_stack' (pcs, GLO, None),
 'operation_sythesis' (kilogram, GLO, None)]

<IPython.core.display.Javascript object>

# Setup LCA Calc 

## Setup functional unit 

In [120]:
# find activity name
def find_by_name(name_ls, database):
    possibles = []
    for activity in database:
        if activity["name"] in name_ls:
            possibles.append(activity)
        else:
            pass

    return possibles

<IPython.core.display.Javascript object>

In [121]:
foreground_name = [
    "manufacturing",
    "operation_input",
    "operation_output",
    "disposal",
]

green_foreground_name = [
    "manufacturing_BoP",
    "manufacturing_stack",
    "manufacturing_nitrogen",
    "manufacturing_ammonia",
    "operation",
]

<IPython.core.display.Javascript object>

In [122]:
def fu(database, foreground_name=foreground_name):
    foreground = find_by_name(foreground_name, database)
    fu_ls = [{foreground[x]: 1} for x in range(len(foreground))]
    return fu_ls

<IPython.core.display.Javascript object>

In [123]:
# create functional unit for ammonia: grey and blue ammonia
CG_FU = fu(database=CG_import)
CG_CCS_FU = fu(database=CG_CCS_import)
SMR_FU = fu(database=SMR_import)
SMR_CCS_FU = fu(database=SMR_CCS_import)
# create functional unit for ammonia: green ammonia
SOEC_Wind_FU = fu(database=SOEC_Wind_import, foreground_name=green_foreground_name)
SOEC_PV_FU = fu(database=SOEC_PV_import, foreground_name=green_foreground_name)
PEM_Wind_FU = fu(database=PEM_Wind_import, foreground_name=green_foreground_name)
PEM_PV_FU = fu(database=PEM_PV_import, foreground_name=green_foreground_name)
AWE_Wind_FU = fu(database=AWE_Wind_import, foreground_name=green_foreground_name)
AWE_PV_FU = fu(database=AWE_PV_import, foreground_name=green_foreground_name)

<IPython.core.display.Javascript object>

## Setup LCIA 

In [125]:
ReCiPe = [method for method in bw.methods if 'ReCiPe 2016 v1.03, midpoint (H)' in str(method) 
                                          and 'w/o LT' not in str(method)
                                          and 'no LT' not in str(method)
                                          and 'obsolete' not in str(method)]
ReCiPe_end = [method for method in bw.methods if 'ReCiPe 2016 v1.03, endpoint (H)' in str(method)
                                          and 'no LT' not in str(method)
                                          and 'total' in str(method)
                                                 ] # point based total 
ReCiPe


[('ReCiPe 2016 v1.03, midpoint (H)',
  'acidification: terrestrial',
  'terrestrial acidification potential (TAP)'),
 ('ReCiPe 2016 v1.03, midpoint (H)',
  'climate change',
  'global warming potential (GWP1000)'),
 ('ReCiPe 2016 v1.03, midpoint (H)',
  'ecotoxicity: freshwater',
  'freshwater ecotoxicity potential (FETP)'),
 ('ReCiPe 2016 v1.03, midpoint (H)',
  'ecotoxicity: marine',
  'marine ecotoxicity potential (METP)'),
 ('ReCiPe 2016 v1.03, midpoint (H)',
  'ecotoxicity: terrestrial',
  'terrestrial ecotoxicity potential (TETP)'),
 ('ReCiPe 2016 v1.03, midpoint (H)',
  'energy resources: non-renewable, fossil',
  'fossil fuel potential (FFP)'),
 ('ReCiPe 2016 v1.03, midpoint (H)',
  'eutrophication: freshwater',
  'freshwater eutrophication potential (FEP)'),
 ('ReCiPe 2016 v1.03, midpoint (H)',
  'eutrophication: marine',
  'marine eutrophication potential (MEP)'),
 ('ReCiPe 2016 v1.03, midpoint (H)',
  'human toxicity: carcinogenic',
  'human toxicity potential (HTPc)'),
 ('R

<IPython.core.display.Javascript object>

In [126]:
indicators = [ReCiPe[0], ReCiPe[1], ReCiPe[5], ReCiPe[14]]
indicators

[('ReCiPe 2016 v1.03, midpoint (H)',
  'acidification: terrestrial',
  'terrestrial acidification potential (TAP)'),
 ('ReCiPe 2016 v1.03, midpoint (H)',
  'climate change',
  'global warming potential (GWP1000)'),
 ('ReCiPe 2016 v1.03, midpoint (H)',
  'energy resources: non-renewable, fossil',
  'fossil fuel potential (FFP)'),
 ('ReCiPe 2016 v1.03, midpoint (H)',
  'particulate matter formation',
  'particulate matter formation potential (PMFP)')]

<IPython.core.display.Javascript object>

In [127]:
ia_cat = [
    "TAP",
    "GWP100",
    "FETP",
    "METP",
    "TETP",
    "FFP",
    "FEP",
    "MEP",
    "HTPc",
    "HTPnc",
    "IRP",
    "LOP",
    "SOP",
    "ODPinfinite",
    "PMFP",
    "HOFP",
    "EOFP",
    "WCP",
]
ia_cat1 = ["TAP", "GWP100", "FFP", "PMFP"]

<IPython.core.display.Javascript object>

## Calc LCIA results 

In [128]:
# setup FU and impact assessment method and calculate results
def LCIA_results(FU_ls, ia=ReCiPe, index_ls=foreground_name):
    bw.calculation_setups["FC_LCIA"] = {"inv": FU_ls, "ia": ia}
    multi_LCA = bw.MultiLCA("FC_LCIA").results
    LCIA_results = pd.DataFrame(
        data=multi_LCA, columns=ia, index=[list(i.keys())[0]["name"] for i in FU_ls]
    )
    return   LCIA_results


<IPython.core.display.Javascript object>

In [129]:
# LCIA results
CG_LCIA_results = LCIA_results(CG_FU).reindex(foreground_name)
CG_CCS_LCIA_results = LCIA_results(CG_CCS_FU).reindex(foreground_name)
SMR_LCIA_results = LCIA_results(SMR_FU).reindex(foreground_name)
SMR_CCS_LCIA_results = LCIA_results(SMR_CCS_FU).reindex(foreground_name)

SOEC_Wind_LCIA_results = LCIA_results(SOEC_Wind_FU).reindex(green_foreground_name)
SOEC_PV_LCIA_results = LCIA_results(SOEC_PV_FU).reindex(green_foreground_name)
PEM_Wind_LCIA_results = LCIA_results(PEM_Wind_FU).reindex(green_foreground_name)
PEM_PV_LCIA_results = LCIA_results(PEM_PV_FU).reindex(green_foreground_name)
AWE_Wind_LCIA_results = LCIA_results(AWE_Wind_FU).reindex(green_foreground_name)
AWE_PV_LCIA_results = LCIA_results(AWE_PV_FU).reindex(green_foreground_name)

<IPython.core.display.Javascript object>

In [130]:
pd.concat(
    [
        CG_LCIA_results,
        CG_CCS_LCIA_results,
        SMR_LCIA_results,
        SMR_CCS_LCIA_results,
        SOEC_Wind_LCIA_results,
        SOEC_PV_LCIA_results,
        PEM_Wind_LCIA_results,
        PEM_PV_LCIA_results,
        AWE_Wind_LCIA_results,
        AWE_PV_LCIA_results,
    ],
    axis=0,
    join="outer",
    keys=[
        "CG",
        "CG_CCS",
        "SMR",
        "SMR_CCS",
        "SOEC_Wind",
        "SOEC_PV",
        "PEM_Wind",
        "PEM_PV",
        "AWE_Wind",
        "AWE_PV",
    ],
)

(ReCiPe 2016 v1.03, midpoint (H), acidification: terrestrial, terrestrial acidification potential (TAP))  \
CG        manufacturing                                                5.094664e-04                                                          
          operation_input                                              6.385357e-03                                                          
          operation_output                                             2.996400e-04                                                          
          disposal                                                     3.740855e-05                                                          
CG_CCS    manufacturing                                                5.379840e-04                                                          
          operation_input                                              6.427885e-03                                                          
          operation_output                                             2.996400e-04                                                          
          disposal                                                     3.740855e-05                                                          
SMR       manufacturing                                                5.094664e-04                                                          
          operation_input                                              1.885489e-03                                                          
          operation_output                                             3.600000e-04                                                          
          disposal                                                     6.139323e-07                                                          
SMR_CCS   manufacturing                                                5.205883e-04                                                          
          operation_input                                              1.886940e-03                                                          
          operation_output                                             3.600000e-04                                                          
          disposal                                                     6.139323e-07                                                          
SOEC_Wind manufacturing_BoP                                            2.441105e-05                                                          
          manufacturing_stack                                          2.970153e-04                                                          
          manufacturing_nitrogen                                       4.424793e-06                                                          
          manufacturing_ammonia                                        2.369489e-05                                                          
          operation                                                    8.169320e-04                                                          
SOEC_PV   manufacturing_BoP                                            2.441105e-05                                                          
          manufacturing_stack                                          2.970153e-04                                                          
          manufacturing_nitrogen                                       4.424793e-06                                                          
          manufacturing_ammonia                                        2.369489e-05                                                          
          operation                                                    2.647943e-03                                                          
PEM_Wind  manufacturing_BoP                                            1.752082e-05                                                          
          manufacturing_stack                             

<IPython.core.display.Javascript object>

In [131]:
with ExcelWriter(
    r"D:\2 Master\2 Paper\2 Ammonia\2 Python\4 code\1 data\output\Ammonia_LCIA.xlsx"
) as writer:
    CG_LCIA_results.to_excel(
        writer, sheet_name="CG", encoding="utf-8",
    )
    CG_CCS_LCIA_results.to_excel(
        writer, sheet_name="CG_CCSs", encoding="utf-8",
    )
    SMR_LCIA_results.to_excel(
        writer, sheet_name="SMR", encoding="utf-8",
    )
    SMR_CCS_LCIA_results.to_excel(
        writer, sheet_name="SMR_CCS", encoding="utf-8",
    )
    SOEC_Wind_LCIA_results.to_excel(
        writer, sheet_name="SOEC_Wind", encoding="utf-8",
    )
    SOEC_PV_LCIA_results.to_excel(
        writer, sheet_name="SOEC_PV", encoding="utf-8",
    )
    PEM_Wind_LCIA_results.to_excel(
        writer, sheet_name="PEM_Wind", encoding="utf-8",
    )
    PEM_PV_LCIA_results.to_excel(
        writer, sheet_name="PEM_PV", encoding="utf-8",
    )
    AWE_Wind_LCIA_results.to_excel(
        writer, sheet_name="AWE_Wind", encoding="utf-8",
    )
    AWE_PV_LCIA_results.to_excel(
        writer, sheet_name="AWE_PV", encoding="utf-8",
    )

<IPython.core.display.Javascript object>